In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
sys.path.append(os.path.abspath(os.path.join('..')))
import sys
warnings.filterwarnings('ignore')

In [6]:
# importing scripts

from scripts.data_visualization import Data_Viz 
from scripts.data_cleaning import DataCleaner

DC = DataCleaner()
DV = Data_Viz()

In [7]:
#loading data
train_store_data = pd.read_csv('../data/train_store.csv')
test_store_data = pd.read_csv('../data/test_store.csv')

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263.0,555.0,1,1,0,1,c,a,1270.0,9.0,2008.0,0,Not Available,Not Available,Not Available
1,2,5,2015-07-31,6064.0,625.0,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
